This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

In [1]:
%cd /code

/code


---

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [2]:
from dotenv import load_dotenv
from notebook_helpers import usage_string, mprint

load_dotenv()

True

---

# Chains

A `chain` is an object that executes a sequence of tasks called `links`. Each link in the chain is a callable, which can be either a function or an object that implements the `__call__` method. **The output of one link serves as the input to the next link in the chain. So a `Chain` is a simple mechanism that takes an input and sends the input to the first link, and the propegates the output of the first link to the second link, and so on, until the end of the chain is reached, and returns the final result.** 

Furthermore, a chain aggregates the history (prompts/responses, token usage, costs, etc.) across all links. More specifically, it aggregates all of the `Record` objects across any link that has a `history` property (which returns a list of Record objects; a Record object contains the metadata of an event such as costs, tokens used, prompt, response, etc.). This functionality allows the chain to contain convenient properties that aggregate the costs and usage across all links of the chain, and can also be used to explore intermediate steps and events in the chain.

---

## Example - Using a model as a prompt enhancer

Here's an example of a simple "prompt enhancer", where the first model enhances the user's prompt and the second model provides a response based on the enhanced prompt.

- first link: defines a prompt-template that takes the user's prompt, and creates a new prompt asking a chat model to improve the prompt (within the context of creating python code)
- second link: the model that takes the modified prompt and improves the prompt
- third link: the model used for the chat/assistant; takes the response from the last model (which is an improved prompt) and returns the request
- fourth link: ignores the response from the chat model, creates a new prompt asking the chat model to extract the code created in the previous response
- fifth link: chat model, which internally maintains the the history of messages 

In [3]:
from llm_chain.base import Chain
from llm_chain.models import OpenAIChat

prompt_enhancer = OpenAIChat(model_name='gpt-3.5-turbo')
# different model/object, therefore different message history (i.e. conversation)
chat_assistant = OpenAIChat(model_name='gpt-3.5-turbo')

def prompt_template(user_prompt: str) -> str:
    return "Improve the user's request, below, by expanding the request " \
        "to describe the relevant python best practices and documentation " \
        f"requirements that should followed:\n\n```{user_prompt}```"

def prompt_extract_code(_) -> str:
    # `_` signals that we are ignoring the input (from the previous link)
    return "Return only the primary code of interest from the previous answer, "\
        "including docstrings, but without any text/response."

# the only requirement for the list is that each item/link is a callable
# where the output of one link matches the input of the next link
# input to the chain is passed to the first link
# the output of the last link is returned by the chain
chain = Chain(links=[
    prompt_template,      # modifies the user's prompt
    prompt_enhancer,      # returns an improved version of the user's prompt
    chat_assistant,       # returns the chat response based on the improved prompt
    prompt_extract_code,  # prompt to ask the model to extract only the relevant code
    chat_assistant,       # returns only the function from the model's last response
])
prompt = "create a function to replace any whitespace character with a single space"
response = chain(prompt)
print(response)

```python
def replace_whitespace(input_string: str) -> str:
    """
    Replaces any whitespace character in the input string with a single space.

    Args:
        input_string (str): The input string to process.

    Returns:
        str: The processed string with whitespace characters replaced by a single space.

    Raises:
        TypeError: If the input parameter is not a string.

    Examples:
        >>> replace_whitespace("Hello\tworld\n")
        'Hello world '
        >>> replace_whitespace("   multiple   spaces   ")
        ' multiple spaces '

    Additional Considerations:
        - Leading and trailing whitespace characters will be preserved.
        - Multiple consecutive whitespace characters will be replaced by a single space.
    """
    if not isinstance(input_string, str):
        raise TypeError("Input parameter must be a string.")

    return ' '.join(input_string.split())
```


In [4]:
print(f"Cost:            ${chain.cost:.4f}")
print(f"Total Tokens:     {chain.total_tokens:,}")
print(f"Prompt Tokens:    {chain.prompt_tokens:,}")
print(f"Response Tokens:  {chain.response_tokens:,}")

Cost:            $0.0035
Total Tokens:     2,016
Prompt Tokens:    1,142
Response Tokens:  874


We can view the history of the chain (i.e. the aggregated history across all links) with the `chain.history` property. 

In this example, the only class that tracks history is `OpenAIChat`. Therefore, both the `prompt_enhancer` and `chat_assistant` objects will contain history. `chain.history` will return a list of three `ExchangeRecord` objects. The first record corresponds to our request to the `prompt_enhancer`, and the second two records correspond to our `chat_assistant` requests. An ExchangeRecord represents a single exchange/transaction with an LLM, encompassing an input (prompt) and its corresponding output (response), along with other properties like cost and token usage.

In [5]:
[type(x) for x in chain.history]

[llm_chain.base.ExchangeRecord,
 llm_chain.base.ExchangeRecord,
 llm_chain.base.ExchangeRecord]

We can view the response we received from the `prompt_enhancer` model by looking at the first record's `response` property (or the second record's `prompt` property since we pass the output of `prompt_enhancer` as the input to the `chat_assistant`):

In [6]:
mprint(chain.history[0].response)

Please create a Python function that adheres to the following best practices and documentation requirements:

1. Function Name: Choose a descriptive and meaningful name for the function, following the Python naming conventions (lowercase with words separated by underscores).

2. Function Parameters: Define the function to accept a single parameter, which should be a string.

3. Return Type: Specify the return type of the function as a string.

4. Function Description: Provide a clear and concise description of what the function does, including its purpose and expected behavior.

5. Input Validation: Consider adding input validation to ensure that the input parameter is a string. You can use the `isinstance()` function to check if the input is of type `str`.

6. Algorithm: Describe the algorithm or approach used to replace any whitespace character with a single space. Consider using Python's built-in string methods or regular expressions for efficient and concise code.

7. Error Handling: Handle any potential errors or exceptions that may occur during the execution of the function. For example, if the input parameter is not a string, you can raise a `TypeError` with an appropriate error message.

8. Examples: Provide one or more examples of how to use the function, including both valid and invalid inputs, along with the expected output.

9. Additional Considerations: Consider any additional considerations or edge cases that may need to be addressed, such as handling leading/trailing whitespace or multiple consecutive whitespace characters.

By following these best practices and documenting your code effectively, you will create a function that is easier to understand, maintain, and collaborate on with other developers.

We could also view the original response from the `chat_assistant` model:

In [7]:
mprint(chain.history[1].response)

Here is an example of a Python function that adheres to the best practices and documentation requirements mentioned:

```python
def replace_whitespace(input_string: str) -> str:
    """
    Replaces any whitespace character in the input string with a single space.

    Args:
        input_string (str): The input string to process.

    Returns:
        str: The processed string with whitespace characters replaced by a single space.

    Raises:
        TypeError: If the input parameter is not a string.

    Examples:
        >>> replace_whitespace("Hello\tworld\n")
        'Hello world '
        >>> replace_whitespace("   multiple   spaces   ")
        ' multiple spaces '

    Additional Considerations:
        - Leading and trailing whitespace characters will be preserved.
        - Multiple consecutive whitespace characters will be replaced by a single space.
    """
    if not isinstance(input_string, str):
        raise TypeError("Input parameter must be a string.")

    return ' '.join(input_string.split())

```

In this example, the function `replace_whitespace` takes a single parameter `input_string` of type `str` and returns a processed string with whitespace characters replaced by a single space.

The function uses the `isinstance()` function to validate that the input parameter is a string. If it is not, a `TypeError` is raised with an appropriate error message.

The algorithm used to replace whitespace characters is as follows:
1. The `split()` method is used to split the input string into a list of substrings, using whitespace characters as separators.
2. The `join()` method is used to join the substrings in the list with a single space as the separator.

The function includes examples of how to use it, including valid and invalid inputs, along with the expected output. It also mentions additional considerations, such as preserving leading/trailing whitespace and replacing multiple consecutive whitespace characters with a single space.

 The final response returned by the `chat_assistant` (and by the `chain` object) returns only the `replace_whitespace` function. The `response` object should match the `response` value in the last record (`chain.history[-1].response`).

In [8]:
assert response == chain.history[-1].response

---

## Example -- Using the results of a web-search in a chat message.

In the following example, we will ask a chat model a question, and provide the model with additional context based on the most relevant text we find in a web-search. For the sake of the example, we will ask the chat model to the answer it provides and summarize that answer.  The workflow is defined as follows:


|   input   |  output  |   link/task   |
|-----------|----------|----------|
|  `None` |   str    |  Ask a question: e.g. `"What is the meaning of life?"`  |
|    str    |   urls   |  Do a web-search via `DuckDuckGo` based on the question.  |
|   urls    | Documents|  Take the urls, scrape the corresponding web-pages, and convert to a list of `Document` objects  |
| Documents | Documents|  Split the `Document` objects into smaller chunks (also `Document` objects)  |
| Documents | `None` |  Store the Documents in a `Chroma` document index so that we can lookup the most relevant documents |
| `None`  |   str    |  Return the original question `What is the meaning of life?` |
|    str    |   str    |  Take the original question, lookup the most relevant documents, and construct the corresponding prompt with the documents injected into the prompt. |
|    str    |   str    |  Pass the prompt to the chat model; receive a response |
|    str    |   str    |  Construct a new prompt, containing the response and a request to summarize the response. |
|    str    |   str    |  Pass the new prompt to the chat model; receive a response |

**Notice how the output of one link matches the input of the next link. This means you could swap out any of the objects we use below with your own custom function or class and the only requirement is that the input/output matches for that particular link.**



### Initial objects in chain

Let's define the objects and functions we are going to include in the chain. We're including many links/tasks so that we can see how flexible our chain can be.

See the [tools.ipynb](https://github.com/shane-kercheval/llm-chain/tree/main/examples/tools.ipynb) notebook for examples of how the various helper classes below (e.g. `DuckDuckGoSearch`, `scrape_url`, etc.) are used.

#### Quick note on the `Value` object.

The only bit of magic below is that we're using a `Value` object to cache the initial question, feed it into the web-search, and then inject it back into the chain at a later point (passing it to the prompt-template). A `Value` object is a callable that, when called with a value caches and returns the same value, and when called without a value, simply returns the previously cached value. When you understand what it's doing, it's not magic at all. Here's a simple example:

In [9]:
from llm_chain.base import Value
cache = Value()
result = cache("This is a value")  # calling the object with a value caches and returns the value
print(result)  # the value in `result` is whatever was passed in to the Value object
print(cache())  # calling the object without a parameter returns the cached value
result = cache("new value")
print(result)
print(cache())

This is a value
This is a value
new value
new value


In [10]:
import re
from llm_chain.base import Document, Chain, Value
from llm_chain.models import OpenAIEmbedding, OpenAIChat
from llm_chain.tools import DuckDuckGoSearch, scrape_url, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.prompt_templates import DocSearchTemplate

# Seach DuckDuckGo based on initial question (passed into the chain)
duckduckgo_search = DuckDuckGoSearch(top_n=3)

# define a function that takes the links from the web-search, scrapes the web-pages,
# and then creates Document objects from the text of each web-page
def scrape_urls(search_results):
    """
    For each url (i.e. `href` in `search_results`):
    - extracts text
    - replace new-lines with spaces
    - create a Document object
    """
    return [
        Document(content=re.sub(r'\s+', ' ', scrape_url(x['href'])))
        for x in search_results
    ]

# Embeddings model used for document index/search (the documents created from the web-search)
embeddings_model = OpenAIEmbedding(model_name='text-embedding-ada-002')
document_index = ChromaDocumentIndex(embeddings_model=embeddings_model, n_results=3)
# DocSearchTemplate uses the ChromaDocumentIndex object to search for the most relevant documents
# (from the web-search) based on the intitial question (which it takes as an input)
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=3)
# Create a chat model without streaming.
non_streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo')
# Create a chat model with streaming enabled via callback.
streaming_chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    streaming_callback=lambda x: print(x.response, end='|'),
)

# A `Value` object is a simple caching mechanism. It's a callable that, when passed a value, it
# caches and returns that value; and when called without a value, it returns the cached value.
# Below, it's being used to cache the original question, feed the question into the web-search
# (`DuckDuckGoSearch`), and then re-inject the question back in the chain and into the
# prompt-template (`DocSearchTemplate`).
question = Value()
# This simple function takes the response from the original chat model and creates a prompt that
# asks the model to summarize the response.
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'

### Defining and Running the Chain

In [11]:
# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    question,
    duckduckgo_search,
    scrape_urls,
    split_documents,  # split web-pages into smaller chunks; defaults to chunk-size of 500
    document_index,  # __call__ function calls add() if given a list of documents (which is returned by `split_documents`)
    question,
    prompt_template,
    non_streaming_chat,
    question_2,
    streaming_chat,
])
# the value passed into `chain()` is passed to the `initial_question` object (which
# is `Value` object and so it caches the value passed in and also returns it) which then gets
# passed to the `duckduckgo_search` object, and so on.
# the response of the final model is streamed, because our chat model has the streaming_callback
# set, but it should also match the response returned
response = chain("What is langchain?")

Lang|Chain| is| a| platform| with| tools| and| APIs| for| creating| applications| using| Language| Models| (|LL|Ms|).|

In [12]:
mprint("> " + response)

> LangChain is a platform with tools and APIs for creating applications using Language Models (LLMs).

---

### Total costs and usage

The `Chain` object aggregates the costs/usage across all links that have a `history` property where that `history` property returns `UsageRecord` objects. In other words, any link that tracks its own history automatically gets counted towards the total usage within the Chain.

In [13]:
# usage_string simply uses the properties in chain (chain.cost, chain.total_tokens, etc.)
# to create a formatted string
print(usage_string(chain))

Cost:              $0.00182
Total Tokens:       11,975
Prompt Tokens:      391
Response Tokens:    40
Embedding Tokens:   11,544



In [14]:
chain.embedding_tokens

11544

### History

Similar to tracking the costs/usage, we can dig into the history at a more granular level.

As you can see below, the `history` property returns a list containing:

- a `SearchRecord` object capturing our original search query and results
- two `EmbeddingsRecord` records; the first corresponds to getting the embeddings of all of the chunks from the web-pages; the second corresponds to getting the embedding of our original question so that we can find the most relevant chunks; the `EmbeddingsRecord` is a `UsageRecord` so it contains costs/usage
- two `MessageRecord` records; the first corresponds to the first question (prompt & response) we made to the chat model; the second corresponds to our second query to the chat model asking it to summarize the first response; the `MessageRecord` is a `UsageRecord` so it contains costs/usage

In [15]:
[type(x) for x in chain.history]

[llm_chain.tools.SearchRecord,
 llm_chain.base.EmbeddingRecord,
 llm_chain.base.EmbeddingRecord,
 llm_chain.base.ExchangeRecord,
 llm_chain.base.ExchangeRecord]

In [16]:
print(chain.history[3])
mprint('---')
print(chain.history[4])

timestamp: 2023-07-07 00:43:32.185; prompt: "Answer the question at the end of the text as trut..."; response: "LangChain is a platform that provides tools and AP...";  cost: $0.000552; total_tokens: 361; prompt_tokens: 340; response_tokens: 21; uuid: dd21d99d-5e92-45c5-a187-43d7502f0336


---

timestamp: 2023-07-07 00:43:33.126; prompt: "Summarize the following in less than 20 words: "La..."; response: "LangChain is a platform with tools and APIs for cr...";  cost: $0.000114; total_tokens: 70; prompt_tokens: 51; response_tokens: 19; uuid: 27e691f8-ffcc-4894-8750-0d381e6e9547


#### `exchange_history`

We can use the `exchange_history` property which simply returns all of the history items that are of type `ExchangeRecord`

In [17]:
len(chain.exchange_history)

2

In [18]:
print(chain.exchange_history[0])  # same as chain.history[3]
mprint('---')
print(chain.exchange_history[1])  # same as chain.history[4]

timestamp: 2023-07-07 00:43:32.185; prompt: "Answer the question at the end of the text as trut..."; response: "LangChain is a platform that provides tools and AP...";  cost: $0.000552; total_tokens: 361; prompt_tokens: 340; response_tokens: 21; uuid: dd21d99d-5e92-45c5-a187-43d7502f0336


---

timestamp: 2023-07-07 00:43:33.126; prompt: "Summarize the following in less than 20 words: "La..."; response: "LangChain is a platform with tools and APIs for cr...";  cost: $0.000114; total_tokens: 70; prompt_tokens: 51; response_tokens: 19; uuid: 27e691f8-ffcc-4894-8750-0d381e6e9547


#### First prompt to the chat model

You can see below that we used three chunks from our original web-search in the first prompt we sent to ChatGPT.

In [19]:
mprint(chain.exchange_history[0].prompt)


Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
credentials. By incorporating LLMs through LangChain, developers can create applications that provide more natural and context-aware interactions with users, resulting in enhanced user experiences and improved engagement. Frequently Asked Questions Q1. What is LangChain? A1. LangChain is a platform that provides tools and APIs for building applications powered by Language Models (LLMs). It simplifies the integration of LLMs into your projects, enabling you to leverage advanced language

With LangChain you can seamlessly integrate LLMs into your projects, harnessing their extraordinary capabilities. Let’s embark on an exhilarating journey, exploring the captivating features and boundless possibilities that LangChain unveils. LangChain is an advanced platform that provides developers with a seamless and intuitive interface to leverage the power of LLM in their applications. It offers a range of APIs and tools that simplify the integration of LLM into your projects, enabling you

power for various text generation and understanding tasks. What can be done with Langchain? LangChain, with its diverse set of features, offers developers a wide range of possibilities to explore and leverage in their applications. Let’s dive into the key components of LangChain—models, prompts, chains, indexes, and memory and discover what can be accomplished with each. Models Numerous new LLMs are currently emerging. LangChain provides a streamlined interface and integrations for various
```

Here is the question:

```
What is langchain?
```


#### First response from the chat model

In [20]:
mprint("> " + chain.exchange_history[0].response)

> LangChain is a platform that provides tools and APIs for building applications powered by Language Models (LLMs).

#### Second prompt to the chat model

In [21]:
mprint(chain.exchange_history[1].prompt)

Summarize the following in less than 20 words: "LangChain is a platform that provides tools and APIs for building applications powered by Language Models (LLMs)."

#### Second response from the chat model

In [22]:
mprint("> " + chain.exchange_history[1].response)

> LangChain is a platform with tools and APIs for creating applications using Language Models (LLMs).

---